In [129]:
#|default_exp titrations2

In [130]:
#|export
from typing import List, Dict, Any, Optional
from titrations.basics import *

# Concepts


## Dosing Ladder


## Rule
A rule has a `parameter`, `operation`, and a `threshold` such that running the `operation` on `parameter` and `threshold` always returns a boolean value (True or False). This value can be computed by running the method `is_satisfied`.
The inner state of a `Rule` should be independent of any one patient.

## Action
A specific recommendation or action. It has three methods that can be defined for each specific action type: the first returns a text version, the second a clickable button, and the third implements what should be done when the button is clicked. `Action`s are specific to each patient.

## Rule With Actions
Extends `Rule` but also suggests a list of `Action`s depending on whether it is satisfied or not. These are defined by two attributes: `actions_when_satisfied` and `actions_when_not_satisfied`. In addition to `Action`s, these two attributes can hold other Rules with Actions. The method `get_suggested_actions` evaluates the `Rule` and selects the appropriate list of actions. If the list contains other Rules With Actions, these are evaluated recursively until the list only contains `Action`s.

## Titrator
The actual engine running the titration process. Has `checkpoints`, which is a list of `Rule(s)WithActions`. `checkpoints` indicates both initiation and titration rules, but these can be specified separately by defining `inititation_checkpoints` and `titration_checkpoints`.
`Titrator`s are specific to each patient and medication class.

# Dosing Ladder

In [131]:
#|export

class DosingLadder:
    ladder : Dict[str, List[Medication]]

    def __init__(self, ladder_dict : Dict[str, List[Medication]], single_class : bool = True) -> None:
        # TODO: ensure 'subladders' have the same number of steps

        # TODO: ensure each subladder consists of the same ingredient

        if single_class:
            pass # TODO: ensure all medications are of the same class

        self.ladder = ladder_dict
        # self.ingredient = None
        self.med_class = None

    @property
    def ingredients(self) -> List[Ingredient]:
        # TODO this should be okay if the checks in `__init__` are implemented
        return [self.ladder[med_name][0].ingredient for med_name in self.ladder]

    def get_subladder(self, medication : Ingredient | Medication):
        return self.ladder[medication.name]

    def _get_current_step_index(self, current_med : Medication):
        subladder = self.get_subladder(current_med)
        index = next((i for i, med in enumerate(subladder) if med.dose == current_med.dose), None)
        return index
    
    def _is_at_lowest_step(self, current_med : Medication):
        return current_med.dose == self.get_lowest_step(current_med).dose

    def _is_at_highest_step(self, current_med : Medication):
        return current_med.dose == self.get_highest_step(current_med).dose
    
    def get_next_step_up(self, current_med : Medication):
        current_dose_index = self._get_current_step_index(current_med)
        return self.get_subladder(current_med)[current_dose_index + 1]

    def get_next_step_down(self, current_med : Medication):
        current_dose_index = self._get_current_step_index(current_med)
        return self.get_subladder(current_med)[current_dose_index - 1]
    
    def get_lowest_step(self, medication: Ingredient | Medication):
        return self.get_subladder(medication)[0]

    def get_highest_step(self, medication: Ingredient | Medication):
        return self.get_subladder(medication)[-1]
    
    def get_current_medication_for_patient(self, patient : Patient):
        filtered = list(filter(lambda med: med.ingredient in self.ingredients, patient.medications))
        assert len(filtered) <= 1
        return filtered[0] if filtered else None
    
    @property
    def lowest_steps(self):
        return { med_name : self.ladder[med_name][0] for med_name in self.ladder}
    
    @property
    def highest_steps(self):
        return { med_name : self.ladder[med_name][-1] for med_name in self.ladder}

In [132]:
beta_blocker_class = MedicationClass("Beta Blocker")

metoprolol_succinate = Ingredient("metoprolol succinate", beta_blocker_class)
carvedilol = Ingredient("carvedilol", beta_blocker_class)
bisoprolol = Ingredient("bisoprolol", beta_blocker_class)

In [133]:
beta_blocker_ladder = DosingLadder({
    metoprolol_succinate.name: [
        Medication(metoprolol_succinate, "12.5 mg", "PO", "daily"),
        Medication(metoprolol_succinate, "25 mg", "PO", "daily"),
        Medication(metoprolol_succinate, "50 mg", "PO", "daily"),
        Medication(metoprolol_succinate, "100 mg", "PO", "daily"),
    ],
    carvedilol.name: [
        Medication(carvedilol, "3.125 mg", "PO", "BID"),
        Medication(carvedilol, "6.25 mg", "PO", "BID"),
        Medication(carvedilol, "12.5 mg", "PO", "BID"),
        Medication(carvedilol, "25 mg", "PO", "BID"),
    ],
    bisoprolol.name: [
        Medication(bisoprolol, "1.25 mg", "PO", "BID"),
        Medication(bisoprolol, "2.5 mg", "PO", "BID"),
        Medication(bisoprolol, "5 mg", "PO", "BID"),
        Medication(bisoprolol, "10 mg", "PO", "BID"),
    ]
})

In [134]:
beta_blocker_ladder.get_subladder(metoprolol_succinate)

[metoprolol succinate 12.5 mg PO daily,
 metoprolol succinate 25 mg PO daily,
 metoprolol succinate 50 mg PO daily,
 metoprolol succinate 100 mg PO daily]

In [135]:
beta_blocker_ladder.get_highest_step(metoprolol_succinate)

metoprolol succinate 100 mg PO daily

In [136]:
beta_blocker_ladder.get_lowest_step(metoprolol_succinate)

metoprolol succinate 12.5 mg PO daily

In [137]:
beta_blocker_ladder._get_current_step_index(
    Medication(metoprolol_succinate, "25 mg", "PO", "daily")
    )

1

In [138]:
beta_blocker_ladder._is_at_lowest_step(
    Medication(metoprolol_succinate, "12.5 mg", "PO", "daily")
)

True

In [139]:
beta_blocker_ladder._is_at_highest_step(
    Medication(metoprolol_succinate, "25 mg", "PO", "daily")
)

False

In [140]:
beta_blocker_ladder.get_next_step_up(
    Medication(metoprolol_succinate, "25 mg", "PO", "daily")
)

metoprolol succinate 50 mg PO daily

In [141]:
beta_blocker_ladder.get_next_step_down(
    Medication(metoprolol_succinate, "25 mg", "PO", "daily")
)

metoprolol succinate 12.5 mg PO daily

In [142]:
beta_blocker_ladder.lowest_steps

{'metoprolol succinate': metoprolol succinate 12.5 mg PO daily,
 'carvedilol': carvedilol 3.125 mg PO BID,
 'bisoprolol': bisoprolol 1.25 mg PO BID}

In [143]:
beta_blocker_ladder.highest_steps

{'metoprolol succinate': metoprolol succinate 100 mg PO daily,
 'carvedilol': carvedilol 25 mg PO BID,
 'bisoprolol': bisoprolol 10 mg PO BID}

In [144]:
p = Patient(medications=[Medication(metoprolol_succinate, "25 mg", "PO", "daily")])
beta_blocker_ladder.get_current_medication_for_patient(p)

metoprolol succinate 25 mg PO daily

# Rules

In [145]:
#|export
import operator

class Rule:
    parameter : str
    operation : str
    threshold : Any

    operators = {
        "gt": operator.gt,
        "gte": operator.ge,
        "lt": operator.lt,
        "lte": operator.le,
        "eq": operator.eq,
        "neq": operator.ne,
        "in": operator.contains,
    }

    def __init__(self, parameter:str, operation:str, threshold:Any) -> None:
        # assert parameter in VALID_PARAMETERS, "Not a valid parameter"
        assert operation in self.operators, f"Invalid operator {operation}"
        if operation == "in": print(f"Warning: For 'in' operations, it is recommended to use kwargs explicitly.")

        self.parameter = parameter
        self.operation = operation
        self.threshold = threshold

    def _is_satisfied(self, patient : Patient):
        # FIXME: handle 'in' operation more neatly

        patient_value = getattr(patient, self.parameter, None)  
        if patient_value is None:  # FIXME: this should be reviewed again
            if type(self.threshold) == bool: raise ValueError(f"Patient has no attribute `{self.parameter}`.")
            else: return False

        # if self.operation == "in": return self.operators[self.operation](self.threshold, patient_value)
        # else:
        return self.operators[self.operation](patient_value, self.threshold)

    def _get_eval_result_object(self, is_satisfied : bool, patient : Optional[Patient] = None) -> Any:
        return type('RuleEvalResult', (), {
            'rule': self,
            'is_satisfied': is_satisfied,
        })

    def evaluate(self, patient : Patient):
        is_satisfied = self._is_satisfied(patient)
        return self._get_eval_result_object(is_satisfied, patient)
    
    def __repr__(self) -> str:
        return f"{self.parameter} {self.operation} {self.threshold}"

In [146]:
r = Rule('SBP', 'lt', 100)
eval_result = r.evaluate(Patient(SBP=100))
eval_result.is_satisfied

False

In [147]:
on_beta_blocker = Rule(threshold='Beta Blocker', operation='in', parameter='current_med_class_names')

# p = Patient(medications=[Medication(metoprolol_succinate, "25 mg", "PO", "daily")])
p = Patient(medications=[])
on_beta_blocker.evaluate(p).__dict__

mappingproxy({'rule': current_med_class_names in Beta Blocker,
              'is_satisfied': False,
              '__module__': '__main__',
              '__dict__': <attribute '__dict__' of 'RuleEvalResult' objects>,
              '__weakref__': <attribute '__weakref__' of 'RuleEvalResult' objects>,
              '__doc__': None})

In [148]:
metoprolol_succinate.__dict__

{'name': 'metoprolol succinate',
 'med_class': <titrations.basics.MedicationClass at 0x1042ad240>}

In [149]:
#|export

class ConditionalRule(Rule):
    condition : Rule

    def __init__(self, parameter:str, operation:str, threshold:Any, condition:Rule) -> None:
        super().__init__(parameter, operation, threshold)
        self.condition = condition

    def _condition_is_met(self, patient: Patient):
        return self.condition.evaluate(patient).is_satisfied

    def _is_satisfied(self, patient: Patient, condition_is_met: bool):
        return super()._is_satisfied(patient) if condition_is_met else False

    def evaluate(self, patient: Patient):
        condition_is_met = self._condition_is_met(patient)
        is_satisfied = self._is_satisfied(patient, condition_is_met)
        return self._get_eval_result_object(is_satisfied, patient)


In [150]:
no_pacemaker = Rule('has_pacemaker', 'eq', False)

In [151]:
p = Patient(HR=50, has_pacemaker=False)
result = no_pacemaker.evaluate(p)
result.is_satisfied

True

In [152]:
bradycardia = ConditionalRule('HR', 'lt', 60, condition=no_pacemaker)
p = Patient(HR=80, has_pacemaker=False)
bradycardia.evaluate(p).is_satisfied

False

# Actions

In [153]:
#|export

class Action:
    """
    This is a base class for 'Action' classes to build upon.
    """
    def __init__(self,
                 patient : Patient,
                 dosing_ladder : DosingLadder,
                 current_medication : Medication):
        self.patient = patient
        self.dosing_ladder = dosing_ladder
        self.current_medication = current_medication

    def suggest(self):
        pass

    def buttons(self):
        pass

    def perform(self):
        pass

In [154]:
#|export

class Start(Action):
    """
    Start a new medication.
    """
    def __init__(self, patient: Patient, dosing_ladder: DosingLadder, current_medication: Medication):
        super().__init__(patient, dosing_ladder, current_medication)
        self.lowest_steps = self.dosing_ladder.lowest_steps

    def suggest(self):
        med_names = list(self.lowest_steps)
        # TODO: this should only be done if the ladder has 3+ subladders
        return f"Start {', '.join([str(self.lowest_steps[med]) for med in med_names[:-1]])}, or {str(self.lowest_steps[med_names[-1]])}."

    def buttons(self):
        return [str(self.lowest_steps[med]) for med in self.lowest_steps]

    def perform(self):
        # TODO: implement this
        pass

In [155]:
a = Start(p, beta_blocker_ladder, "")
a.suggest()

'Start metoprolol succinate 12.5 mg PO daily, carvedilol 3.125 mg PO BID, or bisoprolol 1.25 mg PO BID.'

In [156]:
a.buttons()

['metoprolol succinate 12.5 mg PO daily',
 'carvedilol 3.125 mg PO BID',
 'bisoprolol 1.25 mg PO BID']

In [157]:
#|export

class DoNotStart(Action):
    """
    Do not start a new medication.
    """
    def suggest(self):
        # TODO: modify to suggest not starting a class
        pass

    def perform(self):
        pass

In [158]:
#|export

class StepUp(Action):
    """
    Step up one dose on the dosing ladder.
    """
    def suggest(self):
        next_step_up = self.dosing_ladder.get_next_step_up(self.current_medication)
        return f"Increase {self.current_medication.name} to {str(next_step_up)}."

    def perform(self):
        pass

In [159]:
#|export

class StepDown(Action):
    """
    Step down one dose on the dosing ladder.
    """
    def suggest(self):
        next_step_down = self.dosing_ladder.get_next_step_down(self.current_medication)
        return f"Decrease {self.current_medication.name} to {str(next_step_down)}."

    def perform(self):
        pass

In [160]:
#|export

class Continue(Action):
    """
    Continue the medication at the same dose.
    """
    def suggest(self):
        return f"Continue {str(self.current_medication)}."

    def perform(self):
        pass

In [161]:
#|export

class Stop(Action):
    """
    Stop the medication.
    """
    def suggest(self):
        return f"Stop {self.current_medication.name}."

    def perform(self):
        pass

In [162]:
#|export

class MarkMaxDose(Action):
    """
    Mark current dose as maxiumum tolerated dose.
    """
    def suggest(self):
        return f"Mark {str(self.current_medication)} as maximum tolerated dose."
    
    def perform(self):
        pass

In [163]:
#|export
class ReportReaction(Action):
    """
    File an adverse reaction record.
    """
    def suggest(self):
        return f"File an adverse reaction to {self.current_medication.name}"

    def perform(self, description="reaction"):
        # In real life, this could be opening a modal for entering adverse reactions instead
        self.patient.reactions.append(
            Reaction(self.current_medication.ingredient, description),
        )

# Rules With Actions

In [164]:
#|export

class RuleWithActions(Rule):
    actions_when_satisfied : List[Action] = []
    actions_when_not_satisfied : List[Action] = []
    default_actions_when_satisfied : List[Action] = []  # class attribute
    default_actions_when_not_satisfied : List[Action] = []  # class attribute
    
    def __init__(self,
                 parameter: str, operation: str, threshold: Any,
                 additional_actions_when_satisfied: List[Action] = [],
                 additional_actions_when_not_satisfied: List[Action] = []) -> None:
        super().__init__(parameter, operation, threshold)
        self.actions_when_satisfied = self.default_actions_when_satisfied + additional_actions_when_satisfied
        self.actions_when_not_satisfied = self.default_actions_when_not_satisfied + additional_actions_when_not_satisfied

    def _get_eval_result_object(self, is_satisfied: bool, patient: Patient) -> Any:
        result = super()._get_eval_result_object(is_satisfied, patient)

        recommended_actions = self.actions_when_satisfied if is_satisfied \
            else self.actions_when_not_satisfied
        
        # Recursively evaluate actions that are rules
        for action in recommended_actions:
            if isinstance(action, RuleWithActions):
                action_result = action.evaluate(patient)
                recommended_actions.remove(action)
                recommended_actions += action_result.recommended_actions

        result.recommended_actions = recommended_actions
        return result

class ConditionalRuleWithActions(ConditionalRule, RuleWithActions):
    pass

In [165]:
#|export
class ClassLimitingRule(RuleWithActions):
    default_actions_when_satisfied = [Stop, ReportReaction]

class TitrationLimitingRule(RuleWithActions):
    default_actions_when_satisfied = [Continue, StepDown, MarkMaxDose]

class NonLimitingRule(RuleWithActions):
    pass

In [166]:
#|export
class ConditionTitrationLimitingRule(ConditionalRule, TitrationLimitingRule):
    def _get_eval_result_object(self, is_satisfied: bool, patient: Patient) -> Any:
        # TODO: this is not a neat solution, will need to think of a better way
        return TitrationLimitingRule._get_eval_result_object(self, is_satisfied, patient)

In [167]:
no_pacemaker = Rule('has_pacemaker', 'eq', False)

hypotension = TitrationLimitingRule('SBP', 'lt', 90)
bradycardia = ConditionTitrationLimitingRule('HR' , 'lt', 60, condition=no_pacemaker)
decompensation = TitrationLimitingRule('decompensated', 'eq', True)
symptoms = TitrationLimitingRule('symptomatic', 'eq', True)
av_block = TitrationLimitingRule('av_block', 'eq', True)

In [168]:
p = Patient(HR=50, has_pacemaker=False)
bradycardia.evaluate(p).__dict__

mappingproxy({'rule': HR lt 60,
              'is_satisfied': True,
              '__module__': '__main__',
              '__dict__': <attribute '__dict__' of 'RuleEvalResult' objects>,
              '__weakref__': <attribute '__weakref__' of 'RuleEvalResult' objects>,
              '__doc__': None,
              'recommended_actions': [__main__.Continue,
               __main__.StepDown,
               __main__.MarkMaxDose]})

In [169]:
severe_bradycardia = RuleWithActions('HR', 'lt', 40, [Stop, ReportReaction])
bradycardia = RuleWithActions('HR', 'lt', 60, [severe_bradycardia, Continue, StepDown, MarkMaxDose])

p = Patient(HR=30, has_pacemaker=False)

eval_result = bradycardia.evaluate(p)
recommended_actions = eval_result.recommended_actions
recommended_actions

[__main__.Continue,
 __main__.StepDown,
 __main__.MarkMaxDose,
 __main__.Stop,
 __main__.ReportReaction]

In [170]:
pre_arrest = RuleWithActions('HR', 'lt', 20, [DoNotStart])
severe_bradycardia = RuleWithActions('HR', 'lt', 40, [pre_arrest, Stop, ReportReaction])
bradycardia = RuleWithActions('HR', 'lt', 60, [severe_bradycardia, Continue, StepDown, MarkMaxDose])

p = Patient(HR=10, has_pacemaker=False)

eval_result = bradycardia.evaluate(p)
recommended_actions = eval_result.recommended_actions
recommended_actions

[__main__.Continue,
 __main__.StepDown,
 __main__.MarkMaxDose,
 __main__.Stop,
 __main__.ReportReaction,
 __main__.DoNotStart]

## MaxTolerated

In [ ]:
#|export
class MaxTolerated(RuleWithActions):
    actions_when_satisfied = [Continue]
    def __init__(self, dosing_ladder : DosingLadder, current_medication : Optional[Medication] = None) -> None:
        self.dosing_ladder = dosing_ladder
        self.current_medication = current_medication

    def _is_satisfied(self, patient: Patient):
        if not self.current_medication: self.current_medication = self.dosing_ladder.get_current_medication_for_patient(patient)
        if self.current_medication and self.current_medication.name in patient.max_tolerated:
            return str(self.current_medication) == str(patient.max_tolerated[self.current_medication.name])
        return False
    
    def __repr__(self) -> str:
        return "Max tolerated dose?"

In [172]:
p = Patient(medications=[Medication(metoprolol_succinate, "25 mg", "PO", "daily")], max_tolerated={"metoprolol succinate": Medication(metoprolol_succinate, "25 mg", "PO", "daily")})
r = MaxTolerated(beta_blocker_ladder)
r.evaluate(p).__dict__

mappingproxy({'rule': Max tolerated dose?,
              'is_satisfied': True,
              '__module__': '__main__',
              '__dict__': <attribute '__dict__' of 'RuleEvalResult' objects>,
              '__weakref__': <attribute '__weakref__' of 'RuleEvalResult' objects>,
              '__doc__': None,
              'recommended_actions': [__main__.Continue]})

In [173]:
#|export
htn_target = RuleWithActions('SBP', 'lt', 130, additional_actions_when_satisfied=[Continue])

# Titrator

In [174]:
#|export
from inspect import isclass
from itertools import chain

class Titrator:
    patient : Patient
    dosing_ladder : DosingLadder

    current_medication : Medication
    
    # class attributes
    default_titration_target : type[Rule] | Rule
    default_rules : List[RuleWithActions]
    default_initiation_rules : List[RuleWithActions]
    default_titration_rules : List[RuleWithActions]
    default_initiation_actions : List[Action] = [Start]
    default_titration_actions : List[Action] = [StepUp]
    
    # instance attributes
    titration_target : Rule
    rules : List[RuleWithActions]
    initiation_rules : List[RuleWithActions]
    titration_rules : List[RuleWithActions]
    initiation_actions : List[Action]
    titration_actions : List[Action]

    can_advance : bool
    satisfied_rules : List[Rule]
    recommended_actions : List[Action]

    def __init__(self, patient : Patient,
                 dosing_ladder : Optional[DosingLadder] = None,
                 titration_target : type[Rule] | Rule = MaxTolerated) -> None:
        self.patient = patient

        if dosing_ladder: self.dosing_ladder = dosing_ladder
        else: assert hasattr(self, 'dosing_ladder'), "Dosing ladder must be specified."

        self.current_medication = self.dosing_ladder.get_current_medication_for_patient(patient)

        if isclass(titration_target):
            self.titration_target = titration_target(self.dosing_ladder, self.current_medication)
        else:
            self.titration_target = titration_target

        # TODO: assert hasattr(self, 'titration_target')
        # assert hasattr(self, 'default_rules') or (hasattr(self, 'default_initiation_rules') and \
        #        hasattr(self, 'default_titration_rules')), "Rules must be specified."
        
        self.rules = self.default_rules + [self.titration_target]

        self.initiation_actions = self.default_initiation_actions  # TODO: allow override
        self.titration_actions = self.default_titration_actions  # TODO: allow override

    @property
    def current_ingredient(self) -> Ingredient:
        return self.current_medication.ingredient if self.current_medication else None

    @property
    def is_initiating(self) -> bool:
        return self.current_medication == None

    @property
    def is_titrating(self) -> bool:
        return not self.is_initiating
    
    def evaluate(self) -> None:
        self._rule_results = map(lambda rule: rule.evaluate(self.patient), self.rules)
        self._results_satisfied = list(filter(lambda result: result.is_satisfied, self._rule_results))
        self.satisfied_rules = list(map(lambda result: result.rule, self._results_satisfied))
        
        self.can_advance = len(self.satisfied_rules) == 0

        if self.can_advance:
            if self. is_initiating:
                self.recommended_actions = [action(self.patient, self.dosing_ladder, self.current_medication) for action in self.initiation_actions]
            else:
                self.recommended_actions = [action(self.patient, self.dosing_ladder, self.current_medication) for action in self.titration_actions]
        else:
            action_lists = [result.recommended_actions for result in self._results_satisfied]
            self.recommended_actions = [action(
                self.patient, self.dosing_ladder, self.current_medication
                ) for action in set(chain.from_iterable(action_lists))]



In [175]:
class BetaBlockerTitrator(Titrator):
    dosing_ladder = beta_blocker_ladder
    default_rules = [
        hypotension,
        bradycardia,
        decompensation,
        symptoms,
        av_block,
    ]

In [176]:
p1 = Patient(SBP=130, HR=70, has_pacemaker=False, decompensated=False, symptomatic=False, av_block=True)
t1 = BetaBlockerTitrator(p)

In [177]:
p2 = Patient(SBP=120, HR=70, has_pacemaker=False, decompensated=False, symptomatic=False, av_block=False,
            medications=[Medication(metoprolol_succinate, "25 mg", "PO", "daily")],
            max_tolerated={"metoprolol succinate": Medication(metoprolol_succinate, "25 mg", "PO", "daily")})
t2 = BetaBlockerTitrator(p)

In [178]:
p3 = Patient(SBP=160, HR=70, has_pacemaker=False, decompensated=False, symptomatic=False, av_block=False,
            medications=[Medication(metoprolol_succinate, "25 mg", "PO", "daily")],
            max_tolerated={"metoprolol succinate": Medication(metoprolol_succinate, "25 mg", "PO", "daily")})
t3 = BetaBlockerTitrator(p3, titration_target=htn_target)
# t3.rules.append(MaxTolerated(beta_blocker_ladder))

In [179]:
t3.evaluate()

In [180]:
t3.can_advance

True

In [181]:
t3.rules

[SBP lt 90,
 HR lt 60,
 decompensated eq True,
 symptomatic eq True,
 av_block eq True,
 SBP lt 130]

In [182]:
t3.titration_target

SBP lt 130

In [183]:
t3.satisfied_rules

[]

In [184]:
t3.recommended_actions

# Export

In [3]:
from nbdev.export import nb_export

nb_export('titrations2.ipynb')